In [1]:
from pathlib import Path
import topagnps2cche1d as t2c

# from t2c import Watershed, CrossSection
# from topagnps2cche1d import Watershed, CrossSection

root_dir = Path("../data_inputs/goodwin_creek_1m/")
# root_dir = Path("../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach")
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"

path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

In [2]:
watershed = t2c.Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()

Reading Reaches: 100%|██████████| 910/910 [10:53<00:00,  1.39it/s]


In [3]:
watershed.ignore_reaches_with_strahler_leq(2)


# watershed.reaches[4].ignore_reach()

# watershed.update_graph()
# watershed.update_junctions_and_node_types()
# watershed.identify_inflow_sources()

# watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
# watershed.set_node_id_to_compute_id()

watershed.update_watershed()

watershed.resample_reaches(id_list="all", step=40)

watershed.assign_cross_section_to_all_points()

RENUMBERING REACHES
Renumbering CCHE1D reach: 1, TopAGNPS reach: 897
Renumbering CCHE1D reach: 2, TopAGNPS reach: 896
Renumbering CCHE1D reach: 3, TopAGNPS reach: 895
Renumbering CCHE1D reach: 4, TopAGNPS reach: 838
Renumbering CCHE1D reach: 5, TopAGNPS reach: 837
Renumbering CCHE1D reach: 6, TopAGNPS reach: 833
Renumbering CCHE1D reach: 7, TopAGNPS reach: 831
Renumbering CCHE1D reach: 8, TopAGNPS reach: 830
Renumbering CCHE1D reach: 9, TopAGNPS reach: 785
Renumbering CCHE1D reach: 10, TopAGNPS reach: 779
Renumbering CCHE1D reach: 11, TopAGNPS reach: 777
Renumbering CCHE1D reach: 12, TopAGNPS reach: 776
Renumbering CCHE1D reach: 13, TopAGNPS reach: 772
Renumbering CCHE1D reach: 14, TopAGNPS reach: 771
Renumbering CCHE1D reach: 15, TopAGNPS reach: 769
Renumbering CCHE1D reach: 16, TopAGNPS reach: 767
Renumbering CCHE1D reach: 17, TopAGNPS reach: 766
Renumbering CCHE1D reach: 18, TopAGNPS reach: 764
Renumbering CCHE1D reach: 19, TopAGNPS reach: 763
Renumbering CCHE1D reach: 20, TopAGNPS 

In [4]:
watershed.plot()

:Overlay
   .NdOverlay.I  :NdOverlay   [Reach_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [Reach_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

In [5]:
df_nodes = watershed.create_cche1d_nodes_df()
df_ch = watershed.create_cche1d_channels_df()
df_lk, df_rc = watershed.create_cche1d_links_and_reaches_df()
df_csec, df_csprf = watershed.create_cche1d_csec_csprf_df()
df_tw = watershed.create_cche1d_twcells_df()
df_inflow_reaches = watershed.get_list_of_inflow_reaches_df()

In [16]:
import importlib

importlib.reload(t2c)

<module 'topagnps2cche1d' from 'C:\\Users\\Luc\\projects\\topagnps2cche1d\\topagnps2cche1d\\__init__.py'>

In [7]:
step = 30
strahler = 2
casename = f"goodwin_creek_1m_strahler_gt{strahler}_step_{step}m"
# step = "native_"
# strahler = 0
# casename = f"ohio_strahler_gt{strahler}_step_{step}m"
watershed.write_cche1d_dat_files(
    casename=casename, output_folder=f"./../data_outputs/{casename}"
)

## Plotting network

In [9]:
watershed.plot(
    frame_height=700, frame_width=900, line_width=2, by="CCHE1D_ID", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [CCHE1D_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [CCHE1D_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

### Cross section

In [10]:
import numpy as np

ws = np.linspace(-10, 10, 15)
zs = 0.02 * ws**2

crosssection = CrossSection(id=1, ws=ws, zs=zs)

In [11]:
ws

array([-10.        ,  -8.57142857,  -7.14285714,  -5.71428571,
        -4.28571429,  -2.85714286,  -1.42857143,   0.        ,
         1.42857143,   2.85714286,   4.28571429,   5.71428571,
         7.14285714,   8.57142857,  10.        ])

In [12]:
crosssection.rbt_idx

15

In [13]:
crosssection.plot(marker_size=7)

:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Scatter.I :Scatter   [x]   (y)